# Machine Learning Foundation

## Section 2, Part e: Regularization LAB


## Learning objectives

By the end of this lesson, you will be able to:

*   Implement data standardization
*   Implement variants of regularized regression
*   Combine data standardization with the train-test split procedure
*   Implement regularization to prevent overfitting in regression problems


In [ ]:
import piplite
await piplite.install(['tqdm', 'seaborn', 'pandas', 'numpy'])

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


# Surpress warnings:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

np.set_printoptions(precision=3, suppress=True)

In the following cell we load the data and define some useful plotting functions.


In [ ]:
np.random.seed(72018)



def to_2d(array):
    return array.reshape(array.shape[0], -1)


    
def plot_exponential_data():
    data = np.exp(np.random.normal(size=1000))
    plt.hist(data)
    plt.show()
    return data
    
def plot_square_normal_data():
    data = np.square(np.random.normal(loc=5, size=1000))
    plt.hist(data)
    plt.show()
    return data

### Loading in Boston Data


In [ ]:
from pyodide.http import pyfetch
 
async def download(url, filename):
    response = await pyfetch(url)
    if response.status == 200:
        with open(filename, "wb") as f:
            f.write(await response.bytes())
path = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML240EN-SkillsNetwork/labs/data/boston_housing_clean.pickle"
 
#you will need to download the dataset; if you are running locally, please comment out the following 
await download(path, "boston_housing_clean.pickle")
 
 
# Import pandas library
import pandas as pd

In [ ]:
with open('boston_housing_clean.pickle', 'rb') as to_read:
    boston = pd.read_pickle(to_read)
boston_data = boston['dataframe']
boston_description = boston['description']

# show the first 5 rows using dataframe.head() method
print("The first 5 rows of the dataframe") 
boston_data.head()

## Data standardization


**Standardizing** data refers to transforming each variable so that it more closely follows a **standard** normal distribution, with mean 0 and standard deviation 1.

The [`StandardScaler`](http://scikit-learn.org/dev/modules/generated/sklearn.preprocessing.StandardScaler.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML240ENSkillsNetwork34171862-2022-01-01#sklearn.preprocessing.StandardScaler) object in SciKit Learn can do this.


**Generate X and y**:


In [ ]:
y_col = "MEDV"

X = boston_data.drop(y_col, axis=1)
y = boston_data[y_col]

**Import, fit, and transform using `StandardScaler`**


In [ ]:
from sklearn.preprocessing import StandardScaler

s = StandardScaler()
X_ss = s.fit_transform(X)

### Exercise:

Confirm standard scaling


In [ ]:
#Hint:

a = np.array([[1, 2, 3], 
              [4, 5, 6]]) 
print(a) # 2 rows, 3 columns

In [ ]:
a.mean(axis=0) # mean along the *columns*

In [ ]:
a.mean(axis=1) # mean along the *rows*

In [ ]:
### BEGIN SOLUTION
X2 = np.array(X)
man_transform = (X2-X2.mean(axis=0))/X2.std(axis=0)
np.allclose(man_transform, X_ss)
### END SOLUTION

### Coefficients with and without scaling


In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

y_col = "MEDV"

X = boston_data.drop(y_col, axis=1)
y = boston_data[y_col]

In [ ]:
lr.fit(X, y)
print(lr.coef_) # min = -18

#### Discussion (together):

The coefficients are on widely different scales. Is this "bad"?


In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
s = StandardScaler()
X_ss = s.fit_transform(X)

In [ ]:
lr2 = LinearRegression()
lr2.fit(X_ss, y)
print(lr2.coef_) # coefficients now "on the same scale"

### Exercise:

Based on these results, what is the most "impactful" feature (this is intended to be slightly ambiguous)? "In what direction" does it affect "y"?

**Hint:** Recall from last week that we can "zip up" the names of the features of a DataFrame `df` with a model `model` fitted on that DataFrame using:

```python
dict(zip(df.columns.values, model.coef_))
```


In [ ]:
### BEGIN SOLUTION
pd.DataFrame(zip(X.columns, lr2.coef_)).sort_values(by=1)
### END SOLUTION

Looking just at the strength of the standardized coefficients LSTAT, DIS, RM and RAD are all the 'most impactful'. Sklearn does not have built in statistical signifigance of each of these variables which would aid in making this claim stronger/weaker

### **(Opcional) Detalles de la regularización - Parte 1**  

Este cuaderno revisa las soluciones del **LAB de Regularización**, centrándose en la **estandarización de datos** y la implementación de **regresión regularizada (Lasso y Ridge)** para evitar el sobreajuste.

---

### **1️⃣ Estandarización de Datos**  
- Se trabaja con el **conjunto de datos de Boston**, extraído de un archivo auxiliar.  
- **Estandarización:** Se transforman las variables para que tengan **media 0 y desviación estándar 1**.  
- Se aplica `StandardScaler` de `sklearn` y se verifica su correcta implementación comparándola con una transformación manual en NumPy.  
- **Verificación con `np.allclose()`**: Se confirma que ambas transformaciones (manual y automática) producen valores equivalentes.  

---

### **2️⃣ Regresión Lineal con Datos Escalados y No Escalados**  
- Se entrena un **modelo de regresión lineal** en ambas versiones de los datos (escalados y sin escalar).  
- **Diferencias en coeficientes:**  
  - **Sin escalar:** Los coeficientes tienen **magnitudes muy diferentes**, dificultando la interpretación de la importancia de las variables.  
  - **Con escalado:** Los coeficientes están en la **misma escala**, lo que facilita la comparación de su impacto en la variable de salida (`MEDV`).  
- Se identifican las **características más influyentes**:  
  - **LSTAT** (estado socioeconómico bajo) tiene el coeficiente más bajo (impacto negativo).  
  - **RM** (número de habitaciones) tiene el coeficiente más alto (impacto positivo).  

---

### **3️⃣ Interpretación y Aplicación**  
- La **regresión lineal con datos escalados** permite evaluar **la importancia relativa de las características**.  
- Se muestra cómo la estandarización **mejora la interpretación y comparación de coeficientes**.  

📌 **Próximos pasos:** Se implementará la **regresión Lasso**, comparando su rendimiento con datos escalados y sin escalar. 🚀

### Lasso with and without scaling


We discussed Lasso in lecture.

Let's review together:

1.  What is different about Lasso vs. regular Linear Regression?
2.  Is standardization more or less important with Lasso vs. Linear Regression? Why?


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures

#### Create polynomial features


[`PolynomialFeatures`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML240ENSkillsNetwork34171862-2022-01-01)


In [ ]:
pf = PolynomialFeatures(degree=2, include_bias=False,)
X_pf = pf.fit_transform(X)

**Note:** We use `include_bias=False` since `Lasso` includes a bias by default.


In [ ]:
X_pf_ss = s.fit_transform(X_pf)

### Lasso


[`Lasso` documentation](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML240ENSkillsNetwork34171862-2022-01-01)


In [ ]:
las = Lasso()
las.fit(X_pf_ss, y)
las.coef_ 

### Exercise

Compare

*   Sum of magnitudes of the coefficients
*   Number of coefficients that are zero

for Lasso with alpha 0.1 vs. 1.

Before doing the exercise, answer the following questions in one sentence each:

*   Which do you expect to have greater magnitude?
*   Which do you expect to have more zeros?


In [ ]:
### BEGIN SOLUTION
las01 = Lasso(alpha = 0.1)
las01.fit(X_pf_ss, y)
print('sum of coefficients:', abs(las01.coef_).sum() )
print('number of coefficients not equal to 0:', (las01.coef_!=0).sum())

In [ ]:
las1 = Lasso(alpha = 1)
las1.fit(X_pf_ss, y)
print('sum of coefficients:',abs(las1.coef_).sum() )
print('number of coefficients not equal to 0:',(las1.coef_!=0).sum())
### END SOLUTION

With more regularization (higher alpha) we will expect the penalty for higher weights to be greater and thus the coefficients to be pushed down. Thus a higher alpha means lower magnitude with more coefficients pushed down to 0.


### Exercise: $R^2$


Calculate the $R^2$ of each model without train/test split.

Recall that we import $R^2$ using:

```python
from sklearn.metrics import r2_score
```


In [ ]:
### BEGIN SOLUTION
from sklearn.metrics import r2_score
r2_score(y,las.predict(X_pf_ss))
### END SOLUTION

#### Discuss:

Will regularization ever increase model performance if we evaluate on the same dataset that we trained on?


## With train/test split


#### Discuss

Are there any issues with what we've done so far?

**Hint:** Think about the way we have done feature scaling.

Discuss in groups of two or three.


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_pf, y, test_size=0.3, 
                                                    random_state=72018)

In [ ]:
X_train_s = s.fit_transform(X_train)
las.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred = las.predict(X_test_s)
r2_score(y_test, y_pred)

In [ ]:
X_train_s = s.fit_transform(X_train)
las01.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred = las01.predict(X_test_s)
r2_score(y_test, y_pred)

### **(Opcional) Detalles de la regularización - Parte 2**  

Este cuaderno introduce la **regresión Lasso (L1)**, explicando cómo difiere de la regresión lineal y su impacto en la regularización. Se analiza la importancia de la **estandarización**, la influencia del parámetro **α (alpha)** y se evalúa el rendimiento del modelo en datos de entrenamiento y prueba.

---

### **1️⃣ Diferencia entre Lasso y Regresión Lineal**  
- **Lasso agrega una penalización en la función de costo**, basada en la **suma del valor absoluto** de los coeficientes.  
- **Estandarización es clave** en Lasso, ya que sin ella, los coeficientes pueden verse penalizados de manera desigual debido a escalas diferentes en las características.  

---

### **2️⃣ Implementación de Lasso y Comparación de α (Regularización)**  
- Se usa la **versión predeterminada de Lasso (α = 1.0)** y se observa que muchos coeficientes **se reducen a cero**, eliminando características irrelevantes.  
- Se compara con un **α más pequeño (0.1)**:  
  - **α más alto** → **Mayor regularización**, más coeficientes en **cero** y menor complejidad del modelo.  
  - **α más bajo** → **Menor regularización**, más coeficientes activos y mayor complejidad.  
- Se evalúa la **suma de los coeficientes** y el **número de coeficientes distintos de cero**, confirmando que α más alto reduce la magnitud y la cantidad de coeficientes activos.  

---

### **3️⃣ Evaluación con R² y División de Datos**  
- Se mide el **R² (coeficiente de determinación)** para evaluar el rendimiento del modelo.  
- **Mayor regularización (α alto) reduce R²** en el conjunto de entrenamiento, ya que limita la capacidad del modelo de ajustarse demasiado a los datos.  
- Se introduce **train_test_split** para evaluar la capacidad de generalización en datos nuevos:  
  - **α alto (1.0)** → R² en prueba **bajo (0.33)**, indicando que el modelo es demasiado simple (**subajuste**).  
  - **α bajo (0.1)** → R² en prueba **mayor**, mejor ajuste al conjunto de retención.  

---

### **4️⃣ Conclusión**  
- **El balance entre sesgo y varianza es clave**: demasiada regularización (alto α) puede hacer que el modelo **subajuste**, mientras que muy poca regularización puede causar **sobreajuste**.  
- **Próximo paso:** Se explorarán **diferentes valores de α** para optimizar el modelo y encontrar el equilibrio adecuado. 🚀

### Exercise

#### Part 1:

Do the same thing with Lasso of:

*   `alpha` of 0.001
*   Increase `max_iter` to 100000 to ensure convergence.

Calculate the $R^2$ of the model.

Feel free to copy-paste code from above, but write a one sentence comment above each line of code explaining why you're doing what you're doing.

#### Part 2:

Do the same procedure as before, but with Linear Regression.

Calculate the $R^2$ of this model.

#### Part 3:

Compare the sums of the absolute values of the coefficients for both models, as well as the number of coefficients that are zero. Based on these measures, which model is a "simpler" description of the relationship between the features and the target?


In [ ]:
### BEGIN SOLUTION

# Part 1

# Decreasing regularization and ensuring convergence
las001 = Lasso(alpha = 0.001, max_iter=100000)

# Transforming training set to get standardized units
X_train_s = s.fit_transform(X_train)

# Fitting model to training set
las001.fit(X_train_s, y_train)

# Transforming test set using the parameters defined from training set
X_test_s = s.transform(X_test)

# Finding prediction on test set
y_pred = las001.predict(X_test_s)

# Calculating r2 score
print("r2 score for alpha = 0.001:", r2_score(y_test, y_pred))


# Part 2

# Using vanilla Linear Regression
lr = LinearRegression()

# Fitting model to training set
lr.fit(X_train_s, y_train)

# predicting on test set
y_pred_lr = lr.predict(X_test_s)

# Calculating r2 score
print("r2 score for Linear Regression:", r2_score(y_test,y_pred_lr))


# Part 3
print('Magnitude of Lasso coefficients:', abs(las001.coef_).sum())
print('Number of coeffients not equal to 0 for Lasso:', (las001.coef_!=0).sum())

print('Magnitude of Linear Regression coefficients:', abs(lr.coef_).sum())
print('Number of coeffients not equal to 0 for Linear Regression:', (lr.coef_!=0).sum())
### END SOLUTION

## L1 vs. L2 Regularization


As mentioned in the deck: `Lasso` and `Ridge` regression have the same syntax in SciKit Learn.

Now we're going to compare the results from Ridge vs. Lasso regression:


[`Ridge`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML240ENSkillsNetwork34171862-2022-01-01)


In [ ]:
from sklearn.linear_model import Ridge

### Exercise

Following the Ridge documentation from above:

1.  Define a Ridge object `r` with the same `alpha` as `las001`.
2.  Fit that object on `X` and `y` and print out the resulting coefficients.


In [ ]:
### BEGIN SOLUTION
# Decreasing regularization and ensuring convergence
r = Ridge(alpha = 0.001)
X_train_s = s.fit_transform(X_train)
r.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred_r = r.predict(X_test_s)

# Calculating r2 score
r.coef_
### END SOLUTION

In [ ]:
las001 # same alpha as Ridge above

In [ ]:
las001.coef_

In [ ]:
print(np.sum(np.abs(r.coef_)))
print(np.sum(np.abs(las001.coef_)))

print(np.sum(r.coef_ != 0))
print(np.sum(las001.coef_ != 0))

**Conclusion:** Ridge does not make any coefficients 0. In addition, on this particular dataset, Lasso provides stronger overall regularization than Ridge for this value of `alpha` (not necessarily true in general).


In [ ]:
y_pred = r.predict(X_pf_ss)
print(r2_score(y, y_pred))

y_pred = las001.predict(X_pf_ss)
print(r2_score(y, y_pred))

**Conclusion**: Ignoring issues of overfitting, Ridge does slightly better than Lasso when `alpha` is set to 0.001 for each (not necessarily true in general).


# Example: Does it matter when you scale?


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_ss, y, test_size=0.3, 
                                                    random_state=72018)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2_score(y_test, y_pred)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=72018)

In [ ]:
s = StandardScaler()
lr_s = LinearRegression()
X_train_s = s.fit_transform(X_train)
lr_s.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred_s = lr_s.predict(X_test_s)
r2_score(y_test, y_pred)

**Conclusion:** It doesn't matter whether you scale before or afterwards, in terms of the raw predictions, for Linear Regression. However, it matters for other algorithms. Plus, as we'll see later, we can make scaling part of a `Pipeline`.



***

### Machine Learning Foundation (C) 2020 IBM Corporation


### **(Opcional) Detalles de la regularización - Parte 3**  

En esta sección del **LAB de Regularización**, se comparan diferentes valores de **α (alpha) en Lasso**, la diferencia entre **Lasso y Ridge**, y la importancia de la **escalación de datos** en modelos regularizados. También se evalúan los modelos mediante la **puntuación R²** en conjuntos de entrenamiento y prueba.

---

### **1️⃣ Comparación de Lasso con Diferentes Valores de α**  
- Se prueba **Lasso con α = 0.001** y se ajusta el número de iteraciones para garantizar la convergencia.  
- Se aplica **train_test_split**, asegurando que la transformación de datos se ajuste solo al conjunto de entrenamiento y luego se use para transformar el conjunto de prueba.  
- **Resultados:**  
  - **Lasso R² = 0.868**, mejor que **Regresión Lineal R² = 0.855**.  
  - **Lasso reduce la magnitud de los coeficientes y elimina más coeficientes** que la regresión lineal.  
  - Lasso mostró **89 coeficientes activos**, mientras que la regresión lineal tenía **104 coeficientes activos**.  

---

### **2️⃣ Comparación entre Lasso y Ridge**  
- Se prueba **Ridge con α = 0.001**, usando la misma transformación de datos.  
- **Diferencias clave:**  
  - **Ridge no elimina coeficientes**, solo los reduce.  
  - **Lasso tiene más coeficientes en cero**, lo que simplifica el modelo.  
  - **Magnitud de los coeficientes:** Ridge tiene coeficientes más grandes que Lasso.  
- **R² de Ridge fue menor que el de Lasso**, indicando que **Lasso generalizó mejor en este caso**.  

📌 **Conclusión:** **Lasso proporcionó una regularización más fuerte**, eliminando coeficientes irrelevantes y mejorando la generalización.

---

### **3️⃣ Importancia de la Escalación de Datos**  
- Se prueba si escalar antes o después de dividir los datos afecta el rendimiento.  
- **Resultados:**  
  - En **regresión lineal simple**, la puntuación R² **no cambia**, ya que no hay regularización.  
  - En **Lasso o Ridge, la escalación es crucial**, ya que afecta la penalización de los coeficientes.  
- **Conclusión:** Siempre se debe aplicar `fit_transform` **solo en el conjunto de entrenamiento**, y luego usar `transform` en el conjunto de prueba.

---

### **4️⃣ Conclusión del Laboratorio y Curso 2**  
- **Lasso fue más efectivo que Ridge en este caso**, eliminando coeficientes y mejorando la generalización.  
- **La escalación es clave para modelos regularizados**, asegurando que los coeficientes sean penalizados de manera uniforme.  
- **Próximo paso:** **Curso 3**, donde se aplicarán estas técnicas en diferentes modelos.

📌 **La regularización es fundamental para mejorar la generalización de los modelos y evitar el sobreajuste.** 🚀